# Hyperparameter Tuning

In this project we will test different approaches to hyperparameter tuning.

We will use some of the models we've created in the [Text Classification with Neural Networks](https://github.com/j-n-t/natural_language_processing/blob/master/Text%20Classification%20with%20Neural%20Networks.ipynb) notebook and will try to improve their performance.

In order to do that, we will test the following methods:

* **Grid Search**
* **Random Search**

### Step 1

We'll start by using **sklearn's GridSearchCV** on our simplest model, the **Dense classifier**. 

#### 1. Perform initial imports

In [1]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Embedding
#from keras.layers import SpatialDropout1D, Conv1D, GlobalMaxPooling1D
#from keras.layers import LSTM
#from keras.layers.wrappers import Bidirectional
#from keras.callbacks import ModelCheckpoint

from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV

import os

import numpy as np
import pandas as pd

Using TensorFlow backend.


#### 2. Load data

In [2]:
# create function to load data

def load_data(n_unique_words, n_words_to_skip):
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=n_unique_words, 
                                                          skip_top=n_words_to_skip)
    return (x_train, y_train), (x_test, y_test)

In [3]:
n_unique_words = 10000 #number of most frequent words to consider
n_words_to_skip = 50 #number of most frequent words to ignore

(x_train, y_train), (x_test, y_test) = load_data(n_unique_words, n_words_to_skip)

In [4]:
len(x_train[0])

218

#### 3. Preprocess data

In [5]:
# create function to preprocess data

def preprocess_data(x_train, x_test, max_review_length, pad_type, trunc_type):
    x_train = pad_sequences(x_train, maxlen=max_review_length, 
                            padding=pad_type, truncating=trunc_type, value=0)

    x_test = pad_sequences(x_test, maxlen=max_review_length, 
                           padding=pad_type, truncating=trunc_type, value=0)
    
    return x_train, x_test

In [6]:
max_review_length = 100 #maximum review length of 100 words
pad_type = 'pre' #add padding characters to the start of every review < 100 words
trunc_type = 'pre' #remove words from the beginning of every review > 100 words

x_train, x_test = preprocess_data(x_train, x_test, max_review_length, pad_type, trunc_type)

In [7]:
x_train[0]

array([1415,    2,    2,    2,    2,  215,    2,   77,   52,    2,    2,
        407,    2,   82,    2,    2,    2,  107,  117, 5952,    2,  256,
          2,    2,    2, 3766,    2,  723,    2,   71,    2,  530,  476,
          2,  400,  317,    2,    2,    2,    2, 1029,    2,  104,   88,
          2,  381,    2,  297,   98,    2, 2071,   56,    2,  141,    2,
        194, 7486,    2,    2,  226,    2,    2,  134,  476,    2,  480,
          2,  144,    2, 5535,    2,   51,    2,    2,  224,   92,    2,
        104,    2,  226,   65,    2,    2, 1334,   88,    2,    2,  283,
          2,    2, 4472,  113,  103,    2,    2,    2, 5345,    2,  178,
          2])

In [8]:
len(x_train[0])

100

#### 4. Design and compile model

In [9]:
def create_model(dropout=0.5, n_dense=64):

    model = Sequential(name='model_dense')
    
    model.add(Embedding(10000, 64, input_length=100))
    model.add(Flatten())
    
    model.add(Dense(n_dense, activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

#### 5. Create model

In [10]:
model = KerasClassifier(build_fn=create_model, verbose=1)

In our first grid search, we'll try to tune the hyperparameters `epochs` and `batch size`.

#### 6. Define grid search parameters

In [11]:
epochs = [2, 4, 6]
batch_size = [64, 128]

param_grid = {'epochs': epochs, 'batch_size': batch_size}

In [12]:
param_grid

{'epochs': [2, 4, 6], 'batch_size': [64, 128]}

In [13]:
grid_search = GridSearchCV(model, param_grid, cv=3, scoring = 'roc_auc')

#### 7. Perform grid search and print results

In [14]:
%%time

grid_search.fit(x_train, y_train)

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
16666/16666 [==============================] - 7s 395us/step - loss: 0.5391 - accuracy: 0.7074
Epoch 2/2
8334/8334 [==============================] - 0s 49us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
16667/16667 [==============================] - 8s 453us/step - loss: 0.5450 - accuracy: 0.6998
Epoch 2/2
8333/8333 [==============================] - 0s 43us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/6
16667/16667 [==============================] - 3s 202us/step - loss: 0.6267 - accuracy: 0.6389
Epoch 2/6
16667/16667 [==============================] - 3s 196us/step - loss: 0.2872 - accuracy: 0.8867
Epoch 3/6
16667/16667 [==============================] - 3s 187us/step - loss: 0.0932 - accuracy: 0.9751
Epoch 4/6
16667/16667 [==============================] - 3s 191us/step - loss: 0.0171 - accuracy: 0.9980
Epoch 5/6
16667/16667 [==============================] - 3s 189us/step - loss: 0.0047 - accuracy: 0.9999
Epoch 6/6
8333/8333 [==============================] - 0s 30us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
25000/25000 [==============================] - 5s 187us/step - loss: 0.5424 - accuracy: 0.7066
Epoch 2/2
25000/25000 [==============================] - 5s 199us/step - loss: 0.2389 - accuracy: 0.9086
Wall time: 6min 54s


GridSearchCV(cv=3, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001BD051FB208>,
             iid='deprecated', n_jobs=None,
             param_grid={'batch_size': [64, 128], 'epochs': [2, 4, 6]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [16]:
grid_search.best_params_

{'batch_size': 128, 'epochs': 2}

In [17]:
grid_search.best_score_

0.930216994468223

In [27]:
# create function to print results

import tabulate

def grid_search_results(cvres):
    
    columns = []
    table = []
    
    for key in param_grid.keys():

        columns.append(key)
    
    columns.append(grid_search.scoring)

    for mean_score, params in sorted(zip(cvres['mean_test_score'], cvres['params']), reverse=True):
        
        row = []

        for key in param_grid.keys():
            
            row.append(params[key])
        
        row.append(mean_score)
        table.append(row)
            
    print(tabulate.tabulate(table, headers=columns, tablefmt='fancy_grid'))

In [28]:
grid_search_results(grid_search.cv_results_)

╒══════════╤══════════════╤═══════════╕
│   epochs │   batch_size │   roc_auc │
╞══════════╪══════════════╪═══════════╡
│        2 │          128 │  0.930217 │
├──────────┼──────────────┼───────────┤
│        6 │          128 │  0.923687 │
├──────────┼──────────────┼───────────┤
│        4 │          128 │  0.923328 │
├──────────┼──────────────┼───────────┤
│        6 │           64 │  0.922947 │
├──────────┼──────────────┼───────────┤
│        2 │           64 │  0.921492 │
├──────────┼──────────────┼───────────┤
│        4 │           64 │  0.920784 │
╘══════════╧══════════════╧═══════════╛


From this grid search of only two hyperparameters, the best option for the **`batch size` seems to be 128**, while using **2 as the number of `epochs` seems to give a slight better result**.

As an alternative to this printing option with **Tabulate**, we could also have used **Pandas**.

In [29]:
df_cv_results = pd.DataFrame(grid_search.cv_results_['params'], columns = ['epochs', 'batch_size'])

In [30]:
df_cv_results['roc_auc'] = grid_search.cv_results_['mean_test_score']

In [31]:
df_cv_results.sort_values(by=['roc_auc'], ascending=False)

,epochs,batch_size,roc_auc
3,2,128,0.930217
5,6,128,0.923687
4,4,128,0.923328
2,6,64,0.922947
0,2,64,0.921492
1,4,64,0.920784


To remove the index, we have several options. We'll illustrate some examples for the top 5 results.

In [33]:
# Method 1

print(df_cv_results.sort_values(by=['roc_auc'], ascending=False).head().to_string(index=False))

 epochs  batch_size   roc_auc
      2         128  0.930217
      6         128  0.923687
      4         128  0.923328
      6          64  0.922947
      2          64  0.921492


In [34]:
# Method 2

from IPython.display import display, HTML
display(HTML(df_cv_results.sort_values(by=['roc_auc'], ascending=False).head().to_html(index=False)))

epochs,batch_size,roc_auc
2,128,0.930217
6,128,0.923687
4,128,0.923328
6,64,0.922947
2,64,0.921492


In [35]:
# Method 3

df_cv_results.sort_values(by=['roc_auc'], ascending=False).head().style.hide_index()

epochs,batch_size,roc_auc
2,128,0.930217
6,128,0.923687
4,128,0.923328
6,64,0.922947
2,64,0.921492


We can now try to to tune some other hyperparameteres, like `n_dense` and `dropout`.

#### 8. Create model

In [53]:
model = KerasClassifier(build_fn=create_model, epochs=2, batch_size=128, verbose=1)

#### 9. Define grid search parameters: `n_dense` and `dropout`

In [54]:
n_dense = [64, 128]
dropout = [0.2, 0.5]

param_grid = {'n_dense': n_dense, 'dropout': dropout}

In [55]:
param_grid

{'n_dense': [64, 128], 'dropout': [0.2, 0.5]}

In [56]:
grid_search = GridSearchCV(model, param_grid, cv=3, scoring = 'roc_auc')

#### 10. Perform grid search and print results

In [57]:
%%time

grid_search.fit(x_train, y_train)

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
16666/16666 [==============================] - 3s 199us/step - loss: 0.6000 - accuracy: 0.6538
Epoch 2/2
8334/8334 [==============================] - 0s 28us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
16667/16667 [==============================] - 3s 208us/step - loss: 0.5773 - accuracy: 0.6800
Epoch 2/2
8333/8333 [==============================] - 0s 28us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
16667/16667 [==============================] - 4s 253us/step - loss: 0.5835 - accuracy: 0.6734
Epoch 2/2
8333/8333 [==============================] - 0s 37us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
25000/25000 [==============================] - 5s 190us/step - loss: 0.5539 - accuracy: 0.6966
Epoch 2/2
25000/25000 [==============================] - 5s 183us/step - loss: 0.2552 - accuracy: 0.9024
Wall time: 1min 49s


GridSearchCV(cv=3, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001BD2E5C76C8>,
             iid='deprecated', n_jobs=None,
             param_grid={'dropout': [0.2, 0.5], 'n_dense': [64, 128]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [58]:
grid_search.best_params_

{'dropout': 0.5, 'n_dense': 64}

In [59]:
grid_search.best_score_

0.9291520313337686

In [60]:
grid_search_results(grid_search.cv_results_)

╒═══════════╤═══════════╤═══════════╕
│   n_dense │   dropout │   roc_auc │
╞═══════════╪═══════════╪═══════════╡
│        64 │       0.5 │  0.929152 │
├───────────┼───────────┼───────────┤
│        64 │       0.2 │  0.927046 │
├───────────┼───────────┼───────────┤
│       128 │       0.5 │  0.92592  │
├───────────┼───────────┼───────────┤
│       128 │       0.2 │  0.920134 │
╘═══════════╧═══════════╧═══════════╛


This time around, we obtain our best result with **64 neurons for `n_dense`** and a **`dropout` value of 0.5**.

Like we did before, we can also use Pandas to print this table.

In [63]:
df_cv_results = pd.DataFrame(grid_search.cv_results_['params'], columns = ['n_dense', 'dropout'])

df_cv_results['roc_auc'] = grid_search.cv_results_['mean_test_score']

df_cv_results.sort_values(by=['roc_auc'], ascending=False).style.hide_index()
# why the change in the number of decimal places in the dropout column?!

n_dense,dropout,roc_auc
64,0.500000,0.929152
64,0.200000,0.927046
128,0.500000,0.925920
128,0.200000,0.920134


Interestingly enough, the format of the dropout column is changed. We can easily correct that.

In [64]:
format_dict = {'dropout': '{:.1f}'}
df_cv_results.sort_values(by=['roc_auc'], ascending=False).style.format(format_dict).hide_index()

n_dense,dropout,roc_auc
64,0.5,0.929152
64,0.2,0.927046
128,0.5,0.925920
128,0.2,0.920134


Even though we are using a very simple model and have only tried to tune 2 hyperparameters at a time with 2 or 3 values for each, we can already see that this process can be somewhat time consuming.

Not only that, but it also makes sense to tune all the hyperparameters together because they are not independent from one another in the way they affect the model's performance. As a side effect, the time to perform this kind of grid search will increase tremendously.

Let's see what happens when we tune all the hyperparameters in one go.

#### 11. Create model

In [44]:
model = KerasClassifier(build_fn=create_model, verbose=1)

#### 12. Define grid search parameters: `epochs`, `batch_size`, `n_dense` and `dropout`

In [45]:
epochs = [2, 4, 6]
batch_size = [64, 128]

n_dense = [64, 128]
dropout = [0.2, 0.5]

param_grid = {'epochs': epochs, 'batch_size': batch_size, 
              'n_dense': n_dense, 'dropout': dropout}

In [46]:
param_grid

{'epochs': [2, 4, 6],
 'batch_size': [64, 128],
 'n_dense': [64, 128],
 'dropout': [0.2, 0.5]}

In [47]:
grid_search = GridSearchCV(model, param_grid, cv=3, scoring = 'roc_auc')

#### 13. Perform grid search and print results

In [48]:
%%time

grid_search.fit(x_train, y_train)

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
16666/16666 [==============================] - 6s 358us/step - loss: 0.5222 - accuracy: 0.7215
Epoch 2/2
8334/8334 [==============================] - 0s 34us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
16667/16667 [==============================] - 5s 313us/step - loss: 0.5074 - accuracy: 0.7317
Epoch 2/2
8333/8333 [==============================] - 0s 36us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/6
16667/16667 [==============================] - 4s 254us/step - loss: 0.5754 - accuracy: 0.6782
Epoch 2/6
16667/16667 [==============================] - 4s 255us/step - loss: 0.2390 - accuracy: 0.9071
Epoch 3/6
16667/16667 [==============================] - 4s 245us/step - loss: 0.0524 - accuracy: 0.9888
Epoch 4/6
16667/16667 [==============================] - 4s 241us/step - loss: 0.0074 - accuracy: 0.9997
Epoch 5/6
16667/16667 [==============================] - 4s 246us/step - loss: 0.0024 - accuracy: 1.0000
Epoch 6/6
8333/8333 [==============================] - 0s 37us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
25000/25000 [==============================] - 5s 189us/step - loss: 0.5388 - accuracy: 0.7048
Epoch 2/2
25000/25000 [==============================] - 5s 181us/step - loss: 0.2422 - accuracy: 0.9074
Wall time: 26min 51s


GridSearchCV(cv=3, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001BD1CE68548>,
             iid='deprecated', n_jobs=None,
             param_grid={'batch_size': [64, 128], 'dropout': [0.2, 0.5],
                         'epochs': [2, 4, 6], 'n_dense': [64, 128]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [49]:
grid_search.best_params_

{'batch_size': 128, 'dropout': 0.5, 'epochs': 2, 'n_dense': 64}

In [50]:
grid_search.best_score_

0.9295886151028102

In [51]:
grid_search_results(grid_search.cv_results_)

╒══════════╤══════════════╤═══════════╤═══════════╤═══════════╕
│   epochs │   batch_size │   n_dense │   dropout │   roc_auc │
╞══════════╪══════════════╪═══════════╪═══════════╪═══════════╡
│        2 │          128 │        64 │       0.5 │  0.929589 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │        64 │       0.2 │  0.927909 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │       128 │       0.5 │  0.925998 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        6 │          128 │        64 │       0.5 │  0.924446 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        6 │          128 │        64 │       0.2 │  0.92349  │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        6 │          128 │       128 │       0.5 │  0.92344  │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        6 │           64 │        64 │ 

Tuning all the hyperparameters at the same time, we get the best results for **2 `epochs`** with a **`batchsize`of 128**, **64 neurons as `n_dense`** and **0.5 as our `dropout`** value.

However, even for a relatively simple model and only two or three options for each of the four hyperparameters we've decided to tune (and with a low cross-validation value of 3), this grid search took more than 25 minutes to complete.

One alternative is to use all the cores of our CPU to speed up the process.

In [52]:
import multiprocessing

# number of cores
multiprocessing.cpu_count()

4

Since we have 4 cores, we could set our parameter `n_jobs=4`. Alternatively, we could set it to `n_jobs=-1`, and it will assume the number of available cores of the machine.

We just needed to define our new `grid_search` variable and start the grid search.

In [ ]:
grid_search = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, scoring = 'roc_auc')

In [ ]:
%%time

grid_search.fit(x_train, y_train)

However this raises 2 problems:

* the full progress of the search does not output to the notebook itself but only to the terminal window
* using this parameter may also interfere with the main neural network training process

With this in mind, we've decided not to use this option.

Another alternative is to perform a **random search** instead. This will be our **step 2**.

### Step 2

#### 1. Perform necessary imports

In [75]:
from sklearn.model_selection import RandomizedSearchCV

#import scipy.stats as ss

#### 2. Define random search parameters

In [76]:
epochs = [2, 4, 6]
batch_size = [64, 128]

n_dense = [64, 128]
dropout = [0, 0.1, 0.2, 0.3, 0.4, 0.5]

param_grid = {'epochs': epochs, 'batch_size': batch_size, 
              'n_dense': n_dense, 'dropout': dropout}

In [77]:
param_grid

{'epochs': [2, 4, 6],
 'batch_size': [64, 128],
 'n_dense': [64, 128],
 'dropout': [0, 0.1, 0.2, 0.3, 0.4, 0.5]}

In [78]:
rand_search = RandomizedSearchCV(model, param_grid, cv=5, scoring = 'roc_auc')

We've increased the number of possibilities for our hyperparameters.

For more solid results, we will use a **cross-validation value of 5** and will use the default value for the number of iterations, `n_iter=10`.

#### 3. Perform random search and print results

In [79]:
%%time

rand_search.fit(x_train, y_train)

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
20000/20000 [==============================] - 4s 187us/step - loss: 0.5652 - accuracy: 0.6872
Epoch 2/4
20000/20000 [==============================] - 4s 178us/step - loss: 0.2334 - accuracy: 0.9104
Epoch 3/4
20000/20000 [==============================] - 4s 185us/step - loss: 0.0522 - accuracy: 0.9883
Epoch 4/4
5000/5000 [==============================] - 0s 31us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
20000/20000 [==============================] - 4s 191us/step - loss: 0.5578 - accuracy: 0.6888
Epoch 2/4
20000/20000 [==============================] - 4s 198us/step - loss: 0.2244 - accuracy: 0.9136
Epoch 3/4
20000/20000 [==============================] - 4s 206us/step - loss: 0.0475 - accuracy: 0.9901
Epoch 4/4
5000/5000 [==============================] - 0s 31us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/6
20000/20000 [==============================] - 4s 182us/step - loss: 0.5502 - accuracy: 0.6945
Epoch 2/6
20000/20000 [==============================] - 4s 182us/step - loss: 0.2182 - accuracy: 0.9177
Epoch 3/6
20000/20000 [==============================] - 4s 180us/step - loss: 0.0419 - accuracy: 0.9923
Epoch 4/6
20000/20000 [==============================] - 4s 187us/step - loss: 0.0051 - accuracy: 0.9999
Epoch 5/6
20000/20000 [==============================] - 4s 183us/step - loss: 0.0016 - accuracy: 1.00001s
Epoch 6/6
5000/5000 [==============================] - 0s 34us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
25000/25000 [==============================] - 5s 181us/step - loss: 0.5043 - accuracy: 0.7360
Epoch 2/2
25000/25000 [==============================] - 5s 184us/step - loss: 0.2073 - accuracy: 0.9220
Wall time: 18min 30s


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001BD2E5C76C8>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'batch_size': [64, 128],
                                        'dropout': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
                                        'epochs': [2, 4, 6],
                                        'n_dense': [64, 128]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=0)

In [80]:
rand_search.best_params_

{'n_dense': 64, 'epochs': 2, 'dropout': 0.1, 'batch_size': 128}

In [81]:
rand_search.best_score_

0.9270258352674942

In [82]:
# this function should have a different name...

grid_search_results(rand_search.cv_results_)

╒══════════╤══════════════╤═══════════╤═══════════╤═══════════╕
│   epochs │   batch_size │   n_dense │   dropout │   roc_auc │
╞══════════╪══════════════╪═══════════╪═══════════╪═══════════╡
│        2 │          128 │        64 │       0.1 │  0.927026 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        4 │          128 │        64 │       0.3 │  0.925024 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        6 │          128 │        64 │       0.1 │  0.924337 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        6 │           64 │        64 │       0.5 │  0.924077 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │       128 │       0.3 │  0.923023 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        4 │          128 │       128 │       0.5 │  0.922457 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        6 │           64 │       128 │ 

From this first run, we can reasonably assume that the best values for **`batch_size`** and **`n_dense`** are, respectively, **128** and **64**.

We will now keep the same possible values for the number of `epochs` and try slightly different values for the `dropout` value.

#### 4. Define random search parameters

In [85]:
# obtain 10 values sampled from a uniform distribution between 0.1 and 0.6 ([loc, loc + scale])

#ss.uniform(loc=0.1, scale=0.5).rvs(10)

In [86]:
epochs = [2, 4, 6]
batch_size = [128]

n_dense = [64]
dropout = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
# dropout = ss.uniform(loc=0.1, scale=0.5)

param_grid = {'epochs': epochs, 'batch_size': batch_size, 
              'n_dense': n_dense, 'dropout': dropout}

In [87]:
rand_search = RandomizedSearchCV(model, param_grid, cv=5, scoring = 'roc_auc')

#### 5. Perform random search and print results

In [88]:
%%time

rand_search.fit(x_train, y_train)

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
20000/20000 [==============================] - 4s 208us/step - loss: 0.5832 - accuracy: 0.6678
Epoch 2/4
20000/20000 [==============================] - 4s 178us/step - loss: 0.2554 - accuracy: 0.8998
Epoch 3/4
20000/20000 [==============================] - 4s 187us/step - loss: 0.0743 - accuracy: 0.9805
Epoch 4/4
5000/5000 [==============================] - 0s 37us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
20000/20000 [==============================] - 4s 185us/step - loss: 0.5800 - accuracy: 0.6762
Epoch 2/4
20000/20000 [==============================] - 4s 191us/step - loss: 0.2603 - accuracy: 0.8973
Epoch 3/4
20000/20000 [==============================] - 4s 198us/step - loss: 0.0793 - accuracy: 0.9799
Epoch 4/4
5000/5000 [==============================] - 0s 31us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
20000/20000 [==============================] - 4s 192us/step - loss: 0.5484 - accuracy: 0.7053
Epoch 2/4
20000/20000 [==============================] - 4s 194us/step - loss: 0.2205 - accuracy: 0.9148
Epoch 3/4
20000/20000 [==============================] - 4s 187us/step - loss: 0.0452 - accuracy: 0.9911
Epoch 4/4
5000/5000 [==============================] - 0s 31us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
25000/25000 [==============================] - 5s 188us/step - loss: 0.5300 - accuracy: 0.7145
Epoch 2/2
25000/25000 [==============================] - 5s 195us/step - loss: 0.2381 - accuracy: 0.9091
Wall time: 13min 22s


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001BD2E5C76C8>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'batch_size': [128],
                                        'dropout': [0.1, 0.2, 0.3, 0.4, 0.5,
                                                    0.6],
                                        'epochs': [2, 4, 6], 'n_dense': [64]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=0)

In [89]:
rand_search.best_params_

{'n_dense': 64, 'epochs': 2, 'dropout': 0.5, 'batch_size': 128}

In [90]:
rand_search.best_score_

0.9318239050957734

In [91]:
# this function should have a different name...

grid_search_results(rand_search.cv_results_)

╒══════════╤══════════════╤═══════════╤═══════════╤═══════════╕
│   epochs │   batch_size │   n_dense │   dropout │   roc_auc │
╞══════════╪══════════════╪═══════════╪═══════════╪═══════════╡
│        2 │          128 │        64 │       0.5 │  0.931824 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │        64 │       0.4 │  0.929686 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │        64 │       0.2 │  0.92927  │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │        64 │       0.1 │  0.927133 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        6 │          128 │        64 │       0.6 │  0.925528 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        6 │          128 │        64 │       0.2 │  0.924949 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        6 │          128 │        64 │ 

The value **2** for the number of **`epochs`** seems to be the best.

We can focus only on the `dropout` value.

#### 5. Define random search parameters

In [92]:
epochs = [2]
batch_size = [128]

n_dense = [64]
dropout = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

param_grid = {'epochs': epochs, 'batch_size': batch_size, 
              'n_dense': n_dense, 'dropout': dropout}

In [93]:
rand_search = RandomizedSearchCV(model, param_grid, cv=5, scoring = 'roc_auc')

#### 7. Perform random search and print results

In [94]:
%%time

rand_search.fit(x_train, y_train)

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 7 is smaller than n_iter=10. Running 7 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
20000/20000 [==============================] - 4s 204us/step - loss: 0.5304 - accuracy: 0.7127
Epoch 2/2
5000/5000 [==============================] - 0s 44us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
20000/20000 [==============================] - 4s 186us/step - loss: 0.5460 - accuracy: 0.7032
Epoch 2/2
5000/5000 [==============================] - 0s 31us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
20000/20000 [==============================] - 5s 227us/step - loss: 0.6609 - accuracy: 0.5911
Epoch 2/2
5000/5000 [==============================] - 0s 34us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
25000/25000 [==============================] - 5s 212us/step - loss: 0.6266 - accuracy: 0.6218
Epoch 2/2
25000/25000 [==============================] - 6s 229us/step - loss: 0.3228 - accuracy: 0.8708
Wall time: 5min 47s


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001BD2E5C76C8>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'batch_size': [128],
                                        'dropout': [0.2, 0.3, 0.4, 0.5, 0.6,
                                                    0.7, 0.8],
                                        'epochs': [2], 'n_dense': [64]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=0)

In [95]:
rand_search.best_params_

{'n_dense': 64, 'epochs': 2, 'dropout': 0.8, 'batch_size': 128}

In [96]:
rand_search.best_score_

0.931809343846745

In [97]:
# this function should have a different name...

grid_search_results(rand_search.cv_results_)

╒══════════╤══════════════╤═══════════╤═══════════╤═══════════╕
│   epochs │   batch_size │   n_dense │   dropout │   roc_auc │
╞══════════╪══════════════╪═══════════╪═══════════╪═══════════╡
│        2 │          128 │        64 │       0.8 │  0.931809 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │        64 │       0.6 │  0.931699 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │        64 │       0.7 │  0.931128 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │        64 │       0.4 │  0.929987 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │        64 │       0.5 │  0.929799 │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │        64 │       0.3 │  0.92869  │
├──────────┼──────────────┼───────────┼───────────┼───────────┤
│        2 │          128 │        64 │ 

This time, the value 0.8 for `dropout` seems to allow to achieve the best result. Anyway, all values from 0.4 to 0.8 seem to allow for a good performance of our model.

Even with a random search, this hyperparameter tuning task seems to be quite time consuming.

We can move to the **cloud** to see if we can complete the hyperparameter tuning in less time. With this improvement in computing resources, we can also try to tune **more complex models**. This will be our **step 3**.

### Step 3